In [3]:
import sys, itertools
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import sklearn, time, math
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix

if '../tools' not in sys.path:
    sys.path.append('../tools')
from helper_functions import * 

import warnings
warnings.filterwarnings('ignore')

X, Y = load_all_fv(3, 3)
trX, teX, trY, teY = train_test_split(X, Y, stratify=Y,
                                      random_state=1, test_size=0.1)

Using existing FVs


In [4]:
print('trX {} trY {} teX {} teY {}'.format(trX.shape, trY.shape, teX.shape, teY.shape))

trX (900, 32, 1222) trY (900, 10) teX (100, 32, 1222) teY (100, 10)


In [8]:
import tensorflow as tf
import sys
if '../tools/TensorFlow-Learn/code' not in sys.path:
    sys.path.append('../tools/TensorFlow-Learn/code')
from data_set import DataSet
import regression

train = DataSet(trX, trY)
test = DataSet(teX, teY)

def batchify(t, wildcard=-1):
    t = list(t)
    t[0] = wildcard # why doesn't -1 work?
    return t

# Any batch size on flattened pixel values
x = tf.placeholder("float", shape=batchify(trX.shape, wildcard=None))
y = tf.placeholder("float", shape=batchify(trY.shape, wildcard=None))

x_image = tf.reshape(x, batchify(trX.shape) + [1])

def weight_variable(shape):
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

def bias_variable(shape):
  return tf.Variable(tf.constant(0.1, shape=shape))

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_3x3(x):
  return tf.nn.max_pool(x, ksize=[1, 3, 3, 1],
                        strides=[1, 3, 3, 1], padding='SAME')
# TODO max filter instead?

W_conv1 = weight_variable([5, 50, 1, 4])
b_conv1 = bias_variable([4])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_3x3(h_conv1)

W_conv2 = weight_variable([4, 10, 4, 16])
b_conv2 = bias_variable([16])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_3x3(h_conv2)

W_conv3 = weight_variable([2, 2, 16, 128])
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_3x3(h_conv3)

def max_pool_size(orig, filt):
    if (orig // filt) * filt == orig: return orig // filt
    else: return orig // filt + 1

def layer3(x): return max_pool_size(max_pool_size(max_pool_size(x, 3), 3), 3)

l3size = 128 * layer3(trX.shape[1]) * layer3(trX.shape[2])
W_fc1 = weight_variable([l3size, 1024])
b_fc1 = bias_variable([1024])

h_pool3_flat = tf.reshape(h_pool3, [-1, l3size])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

# Don't drop out when testing by setting keep_prob to 1.0
keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

output = regression.SoftMax(h_fc1_drop, y)

train_step = tf.train.AdamOptimizer(1e-4).minimize(output.cross_entropy)

NUM_CORES = 3 # let me do stuff in the background
sess = tf.Session(config=tf.ConfigProto(inter_op_parallelism_threads=NUM_CORES,
    intra_op_parallelism_threads=NUM_CORES))

with sess.as_default():
    sess.run(tf.initialize_all_variables())

    # TODO add a saver here, this takes forever... Also, time it.
    NUM_EPOCHS = 100
    UPDATE_COARSENESS = 5
    BATCH_SIZE = 50 
    for epoch in range(1, NUM_EPOCHS + 1):
        for i, batch in enumerate(train.new_epoch(BATCH_SIZE), 1):
            tot_batches = train.size // BATCH_SIZE
            two_percent_done = i * 50 // tot_batches
            print(('\rEpoch {:02d}/{:02d} [' + two_percent_done * '-' + (50 - two_percent_done) * ' '
                   + '] {}/{}').format(epoch, NUM_EPOCHS, i, tot_batches), end='')
            sys.stdout.flush()
            train_step.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})
        print('')
        if epoch == NUM_EPOCHS or UPDATE_COARSENESS and epoch % UPDATE_COARSENESS == 0:
            batch = DataSet(*next(train.new_epoch(100)))
            err = batch.multiclass_error(x, output.y, y, feed_dict={keep_prob:1.0})
            print('Epoch {}/{} 100-sample training error {}'.format(epoch, NUM_EPOCHS, err))

    print('Test error {}'.format(test.multiclass_error(x, output.y, y, feed_dict={keep_prob:1.0})))

sess.close()

11776
Epoch 01/100 [--------------------------------------------------] 18/18
Epoch 02/100 [--------------------------------------------------] 18/18
Epoch 03/100 [--------------------------------------------------] 18/18
Epoch 04/100 [--------------------------------------------------] 18/18
Epoch 05/100 [--------------------------------------------------] 18/18
Epoch 5/100 100-sample training error 0.6699999868869781
Epoch 06/100 [--------------------------------------------------] 18/18
Epoch 07/100 [--------------------------------------------------] 18/18
Epoch 08/100 [--------------------------------------------------] 18/18
Epoch 09/100 [--------------------------------------------------] 18/18
Epoch 10/100 [--------------------------------------------------] 18/18
Epoch 10/100 100-sample training error 0.5600000023841858
Epoch 11/100 [--------------------------------------------------] 18/18
Epoch 12/100 [--------------------------------------------------] 18/18
Epoch 13/100 [-